# **Usage Agent**

The **Usage Agent's** functionnality is to estimate the probability that a particular device will be used on the following day. Within the general recommendation framework, this function is used in order to limit unnecessary recommendations that could irritate the user. Whenever a device is unlikely to be used on the next day (estimated likelihood below a certain threshold), no recommendation will be made.

In the present notebook, we will describe how these probabilities are estimated in detail and define the **Usage Agent** class that will be integrated into the recommendation agent.

The Usage Agent will use a ML-algorithm on features extracted from the household's electricity consumption data in order to predict the likelihood of use of devices on the next day. For instance, at a given day t-1, it will use all available consumption data until day t-1 in order to predict device usage on day t. The features we will use can be divided into 3 categories: 
1. Whether activity has been detected in the house in the preceding days (activity detected by electricity consumption).
2. Whether the to-be-prediced-device has been used in the previous days.
3. Time dummies.

Given the limited number of observations for each household, we will need to restrict the complexity of the ML-Algorithm in use. This is the reason why we will use a logit model with a limited number of features.

## **1. Load And Preprocess Data**

This part's only purpose is to load the data used in the Usage Agent. This process is described in detail in the Preparation Agent. 

**Note: When computing the script with another Household than Household 1 you might need to adapt some parameters**

### **1.1 Initialize And Load Python Scripts**

In [ ]:
from helper_functions import Helper
from agents import Preparation_Agent
import numpy as np
# More ML Models
import xgboost as xgb
import sklearn 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
helper = Helper()

In [ ]:
DATA_PATH = '../data/'

In [ ]:
# load household data
household = helper.load_household(DATA_PATH, 1)
active_appliances = ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site']

### **1.2 Set Parameters For Pre-processing Step**

In [ ]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}
aggregate_params24_H = {
    'resample_param': '24H'
}


activity_params = {
    'active_appliances': ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site'],
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

shiftable_devices = {'Dishwasher', 'Washing Machine'}

device = {
    'threshold' : .15}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}



### **1.3 Pre-process Data For Input In Device_Usage Agent**

In [ ]:
# calling the preparation pipeline
prep = Preparation_Agent(household)
df = prep.pipeline_usage(household, activity_pipe_params)
df.to_pickle('../data/processed_pickle/trunctuated.pkl')

#display all potential variables for predicting device usage likelihood
df.head()

## **2.  Constructing the Usage Agent**

### **2.1 Initialize Agent**

First we define the **Usage Agent class**. It takes as input the data generated by the prep.pipeline_usage function computed above, and the name of the device for which predictions should be made (e.g "Washing Machine", "Dishwasher"etc...).

In [ ]:
class Usage_Agent:
    import pandas as pd

    def __init__(self, input_df, device):
        self.input = input_df
        self.device = device

Here we initialize the agent for the device "Dishwasher"

In [ ]:
import pandas as pd
usage = Usage_Agent(df, "Dishwasher") 

### **2.2 Train_test_split function**

The number of data points available to make a prediction for day t increases by one, each time t increases by one. Therefore, we define a custom train_test_split function that automatically puts all data available until day t-1 (incl.) into the training set. The Data for day t (= prediction day) comes into the test set.

In order to limit over-fitting the function also filters out the number of features to be taken into account to train the model. Here these are the following:

1. Indicator of device usage at day t-1.
2. Indicator of device usage at day t-2.
3. Indicator of activity in the household in the past two days.


In [ ]:
#date: the day of prediction
#train start: the day from which training starts
def train_test_split(self, df, date, train_start='2013-11-01'):
    #restrict number of variables
    select_vars =  [self.device + '_usage', self.device+ '_usage_lag_1', self.device+ '_usage_lag_2',	'active_last_2_days']
    df = df[select_vars]
    #spli train and test
    X_train = df.loc[train_start:date, df.columns != self.device + '_usage']
    y_train = df.loc[train_start:date, df.columns == self.device + '_usage']
    X_test  = df.loc[date, df.columns != self.device + '_usage']
    y_test  = df.loc[date , df.columns == self.device + '_usage']
    return X_train, y_train, X_test, y_test

# add to Usage agent
setattr(Usage_Agent, 'train_test_split', train_test_split)
del train_test_split 

In [ ]:
X_train, y_train, X_test, y_test = usage.train_test_split(df, "2014-11-01", train_start='2013-11-01')

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

### **2.3 Fitting Models**

In [ ]:
#### Logit 
def fit_Logit(self, X, y):
    return LogisticRegression(random_state=0).fit(X, y)

#####ML Models ##################
def fit_knn(self, X, y):
     return KNeighborsClassifier(3).fit(X,y)


def fit_random_forest(self, X,y):
    return RandomForestClassifier(max_depth=5, n_estimators=240, max_features=1).fit(X,y)

def fit_ADA(self,X,y):
    return  AdaBoostClassifier().fit(X,y)

def fit_XGB(self, X,y):
    return xgb.XGBClassifier().fit(X,y)
    

# add to Usage agentt
setattr(Usage_Agent, 'fit_Logit', fit_Logit)
del fit_Logit 

setattr(Usage_Agent, 'fit_knn', fit_knn)
del fit_knn

setattr(Usage_Agent, 'fit_random_forest', fit_random_forest)
del fit_random_forest

setattr(Usage_Agent, 'fit_ADA', fit_ADA)
del fit_ADA

setattr(Usage_Agent, 'fit_XGB', fit_XGB)
del fit_XGB


In [ ]:
def fit(self, X, y, model_type):
    model = None
    if model_type == "logit":
        model = self.fit_Logit(X, y)
    elif model_type == "ada":
        model = self.fit_ADA(X,y)
    elif model_type == "knn":
        model = self.fit_knn(X,y)
    elif model_type == "random forest":
        model = self.fit_random_forest(X,y)
    elif model_type == "xgboost":
        model = self.fit_XGB(X,y)
    else:
        raise InputError("Unknown model type.")
    return model

setattr(Usage_Agent, 'fit', fit)
del fit

Now that we have the function to perform the split-sampling we can fit the model on training data. For that purpose, we define a Logit-fitting function as follows:

Using this function on the training split, we can train our first model:

In [ ]:
usage = Usage_Agent(df, "Dishwasher")
model = usage.fit(X_train, y_train, 'logit') # Change it to the model you want to use
print(model)

In [ ]:
test =np.array(X_test).reshape(-1,3)
model.predict_proba(test)

In [ ]:
def predict(self, model, X):
    import numpy as np
    import pandas
    res = 3
    cols = ["temp", "dwpt", "rhum", "wdir", "wspd"]
    for e in cols:
        if isinstance(X, pd.DataFrame):
            if e in X.columns:
                res += 1
        if isinstance(X, pd.Series):
            if e in X.index:
                res += 1
    X = np.array(X).reshape(-1, res)
    if type(model) == sklearn.linear_model.LogisticRegression:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == sklearn.neighbors._classification.KNeighborsClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == sklearn.ensemble._forest.RandomForestClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) ==  sklearn.ensemble._weight_boosting.AdaBoostClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == xgboost.sklearn.XGBClassifier:
        y_hat = model.predict_proba(X)[:,1]
    else:
        raise InputError("Unknown model type.")

    return y_hat

setattr(Usage_Agent, 'predict', predict)
del predict

In [ ]:
def auc(self, y_true, y_hat):
    import sklearn.metrics
    return sklearn.metrics.roc_auc_score(y_true, y_hat)

# add to Activity agent
setattr(Usage_Agent, 'auc', auc)
del auc

In [ ]:
def train_test_split(self, df, date, train_start="2013-11-01"):
        df.columns = df.columns.map(str)
        select_vars = [
            self.device + "_usage",
            self.device + "_usage_lag_1",
            self.device + "_usage_lag_2",
            "active_last_2_days",
        ]
        # Add weather possibly
        if "temp" in df.columns:
            select_vars.append("temp")
            df["temp"].fillna(method="backfill", inplace=True)
        if "dwpt" in df.columns:
            select_vars.append("dwpt")
            df["dwpt"].fillna(method="backfill", inplace=True)
        if "rhum" in df.columns:
            select_vars.append("rhum")
            df["rhum"].fillna(method="backfill", inplace=True)
        if "wdir" in df.columns:
            select_vars.append("wdir")
            df["wdir"].fillna(method="backfill", inplace=True)
        if "wspd" in df.columns:
            select_vars.append("wspd")
            df["wspd"].fillna(method="backfill", inplace=True)

        df = df[select_vars]
        X_train = df.loc[train_start:date, df.columns != self.device + "_usage"]
        y_train = df.loc[train_start:date, df.columns == self.device + "_usage"]
        X_test = df.loc[pd.to_datetime(date), df.columns != self.device + "_usage"]
        y_test = df.loc[pd.to_datetime(date), df.columns == self.device + "_usage"]
        return X_train, y_train, X_test, y_test
    
split_params = {
            "train_start": "2013-11-01",
        }

Once the model is fitted to the training data, a prediction can be made for the test day. This prediction function is defined in the following:

In [ ]:
for local in range(1):
    print(X_test[local])

In [29]:
predict_start="2014-01-01"
predict_end = -1
model_type = 'ada'

In [26]:
X_test.values

array([0., 0., 0.])

In [28]:
from tqdm import tqdm
xai= True
weather_sel= False
return_errors = True
dates = pd.DataFrame(df.index)
dates = dates.set_index(df.index)["Time"]
predict_start = pd.to_datetime(predict_start)
predict_end = (
    pd.to_datetime(dates.iloc[predict_end])
    if type(predict_end) == int
    else pd.to_datetime(predict_end)
)
dates = dates.loc[predict_start:predict_end]
y_true = []
y_hat_train = {}
y_hat_test = []
y_hat_lime = []
y_hat_shap = []
auc_train_dict = {}
auc_test = []
xai_time_lime = []
xai_time_shap = []

predictions_list = []

 
if xai:
    print('The explainability approaches in the Usage Agent are being evaluated for model: ' + str     (model_type))
    print('Start evaluation with LIME and SHAP')
    import time
    import lime
    import shap as shap
    from lime import lime_tabular

    for date in tqdm(dates.index):
        errors = {}
        try:
            X_train, y_train, X_test, y_test = usage.train_test_split(
                df, date, train_start="2013-11-01"
            )
            # fit model
            model = usage.fit(X_train, y_train, model_type)
            # predict
            y_hat_train.update({date: usage.predict(model, X_train)})
            y_hat_test += list(usage.predict(model, X_test))
            # evaluate train data
            auc_train_dict.update(
                {date: usage.auc(y_train, list(y_hat_train.values())[-1])}
            )
            y_true += list(y_test)
            start_time = time.time()

            explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train),
                                                                mode="classification",
                                                                feature_names=X_train.columns,
                                                                categorical_features=[0])

            if model_type == "xgboost":
                import warnings
                warnings.filterwarnings('ignore')
                exp = explainer.explain_instance(X_test.values, model.predict_proba)
            else:
                exp = explainer.explain_instance(data_row=X_test, predict_fn=model.predict_proba)

           

            y_hat_lime += list(exp.local_pred)

            # take time for each day:
            end_time = time.time()
            difference_time = end_time - start_time

            xai_time_lime.append(difference_time)

             # SHAP
            # =========================================================================
            start_time = time.time()

            if model_type == "logit":
                #option: apply kmeans first for faster computation
                X_train_summary = shap.kmeans(X_train, 10)
                explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)
                #without kmeans:
                #explainer = shap.KernelExplainer(model.predict_proba, X_train)

            elif model_type == "ada":
                X_train_summary = shap.kmeans(X_train, 10)
                explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

            elif model_type == "knn":
                X_train_summary = shap.kmeans(X_train, 10)
                explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

            elif model_type == "random forest":
                explainer = shap.TreeExplainer(model, X_train)

            elif model_type == "xgboost":
                explainer = shap.TreeExplainer(model, X_train, model_output='predict_proba')
            else:
                raise InputError("Unknown model type.")

            base_value = explainer.expected_value[1]  # the mean prediction

            
            #for local in range(len(X_test)):  # same as above

            shap_values = explainer.shap_values(
                X_test)
            # hier theoretisch ganzes test set prediction statt for loop möglich
            contribution_to_class_1 = np.array(shap_values).sum(axis=1)[1]  # the red part of the diagram
            shap_prediction = base_value + contribution_to_class_1
            #print(shap_prediction)
            # Prediction from XAI:
            y_hat_shap.append(shap_prediction)
            #print(y_hat_shap)


            # take time for each day:
            end_time = time.time()
            difference_time = end_time - start_time
            xai_time_shap.append(difference_time)
            #print(xai_time_shap)  
        except Exception as e:
            errors[date] = e

auc_test = usage.auc(y_true, y_hat_test)
auc_train = np.mean(list(auc_train_dict.values()))
predictions_list.append(y_true)
predictions_list.append(y_hat_test)
predictions_list.append(y_hat_lime)
predictions_list.append(y_hat_shap)

# Efficiency
time_mean_lime = np.mean(xai_time_lime)
time_mean_shap = np.mean(xai_time_shap)
print('Mean time nedded by appraoches: ' + str(time_mean_lime) + ' ' + str(time_mean_shap))



  0%|          | 0/556 [00:00<?, ?it/s]The explainability approaches in the Usage Agent are being evaluated for model: logit
Start evaluation with LIME and SHAP
100%|██████████| 556/556 [14:16<00:00,  1.54s/it]Mean time nedded by appraoches: 0.796832178565238 0.5186920127561016



In [ ]:
errors

In [ ]:
explainer.explain_instance(data_row=X_test, predict_fn=model.predict_proba).local_pred

In [ ]:
explainer.explain_instance(data_row=X_test,predict_fn=model.predict_proba).local_pred

In [ ]:
xai_time_shap

In [ ]:
df, model_type, train_start, predict_start="2014-01-01", predict_end=-1, return_errors=False, weather_sel=False, xai=False

In [ ]:
def evaluate(
            df, model_type, train_start, predict_start="2014-01-01", predict_end=-1, return_errors=False,
            weather_sel=False, xai=False
    ):
        import pandas as pd
        import numpy as np
        from tqdm import tqdm

        dates = pd.DataFrame(df.index)
        dates = dates.set_index(df.index)["Time"]
        predict_start = pd.to_datetime(predict_start)
        predict_end = (
            pd.to_datetime(dates.iloc[predict_end])
            if type(predict_end) == int
            else pd.to_datetime(predict_end)
        )
        dates = dates.loc[predict_start:predict_end]
        y_true = []
        y_hat_train = {}
        y_hat_test = []
        y_hat_lime = []
        y_hat_shap = []
        auc_train_dict = {}
        auc_test = []
        xai_time_lime = []
        xai_time_shap = []
        
        predictions_list = []

        if weather_sel:
            # Add Weather
            ################################
            from meteostat import Point, Daily
            from datetime import datetime, timedelta

            lough = Point(52.766593, -1.223511)
            time = df.index.to_series(name="time").tolist()
            start = time[0]
            end = time[len(time) - 1]
            weather = Daily(lough, start, end)
            weather = weather.fetch()

            from sklearn.impute import KNNImputer
            import numpy as np

            headers = weather.columns.values

            empty_train_columns = []
            for col in weather.columns.values:
                if sum(weather[col].isnull()) == weather.shape[0]:
                    empty_train_columns.append(col)
            headers = np.setdiff1d(headers, empty_train_columns)

            imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
            weather = imputer.fit_transform(weather)
            scaler = MinMaxScaler()
            weather = scaler.fit_transform(weather)
            weather = pd.DataFrame(weather)
            weather["time"] = time[0:len(weather)]
            df["time"] = time

            weather.columns = np.append(headers, "time")

            df = pd.merge(df, weather, how="right", on="time")
            df = df.set_index("time")
            # df.drop("time", axis=1, inplace=True)
            ################################
        if xai:
            print('The explainability approaches are being evaluated for model: ' + str(model_type))

        
        for date in tqdm(dates.index):
            errors = {}
            try:
                X_train, y_train, X_test, y_test = train_test_split(
                    df, date, train_start
                )
                # fit model
                model = fit(X_train, y_train, model_type)
                # predict
                y_hat_train.update({date: predict(model, X_train)})
                y_hat_test += list(predict(model, X_test))
                # evaluate train data
                auc_train_dict.update(
                    {date: sklearn.metrics.roc_auc_score(y_train, list(y_hat_train.values())[-1])}
                )
                y_true += list(y_test)
                
                if xai:
                    import time
                    import lime
                    from lime import lime_tabular

                    start_time = time.time()

                    if model_type == "xgboost":
                        booster = model.get_booster()

                        explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,
                                                                           feature_names=X_train.columns,
                                                                           kernel_width=3, verbose=False)
                        #print(explainer)


                    else:
                        explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train),
                                                                      mode="classification",
                                                                      feature_names=X_train.columns,
                                                                      categorical_features=[0])

                    # optional to do: only select the instances that are predicted to be 1
                    # for local in
                    #to do:
                    for local in range(2):  # replace 3 with when is works: len(X_test)
                        # to do: hier weiter
                        #print('Instance: ' + str(local))
                        # still predict_proba since also used in other function: treshold dependent outcome
                        if model_type == "xgboost":
                            exp = explainer.explain_instance(X_test[local, :], model.predict_proba)
                        else:
                            exp = explainer.explain_instance(data_row=X_test[local], predict_fn=model.predict_proba)

                        y_hat_lime += list(exp.local_pred)

                    # take time for each day:
                    end_time = time.time()
                    difference_time = end_time - start_time

                    xai_time_lime.append(difference_time)

                    #print('SHAP: ')

                    import shap as shap

                    start_time = time.time()

                    # to do: add for all models
                    if model_type == "logit":
                        #option: apply kmeans first for faster computation
                        X_train_summary = shap.kmeans(X_train, 10)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)
                        #without kmeans:
                        #explainer = shap.KernelExplainer(model.predict_proba, X_train)

                    elif model_type == "ada":
                        X_train_summary = shap.kmeans(X_train, 10)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

                    elif model_type == "knn":
                        X_train_summary = shap.kmeans(X_train, 10)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

                    elif model_type == "random forest":
                        explainer = shap.TreeExplainer(model, X_train)

                    elif model_type == "xgboost":
                        explainer = shap.TreeExplainer(model, X_train, model_output='predict_proba')
                        #print(explainer)
                    else:
                        raise InputError("Unknown model type.")

                    base_value = explainer.expected_value[1]  # the mean prediction

                    #to do:
                    for local in range(len(X_test)):  # replace 3 with when is works: len(X_test)

                        shap_values = explainer.shap_values(
                            X_test[local])
                        # hier theoretisch ganzes test set prediction statt for loop möglich
                        contribution_to_class_1 = np.array(shap_values).sum(axis=1)[1]  # the red part of the diagram
                        shap_prediction = base_value + contribution_to_class_1
                        #print(shap_prediction)
                        # Prediction from XAI:
                        y_hat_shap.append(shap_prediction)
                        #print(y_hat_shap)


                    # take time for each day:
                    end_time = time.time()
                    difference_time = end_time - start_time
                    xai_time_shap.append(difference_time)
                    #print(xai_time_shap)
                
            except Exception as e:
                errors[date] = e

        auc_test = sklearn.metrics.roc_auc_score(y_true, y_hat_test)
        auc_train = np.mean(list(auc_train_dict.values()))
        predictions_list.append(y_true)
        predictions_list.append(y_hat_test)
        predictions_list.append(y_hat_lime)
        predictions_list.append(y_hat_shap)
        
        # Efficiency
        time_mean_lime = np.mean(xai_time_lime)
        time_mean_shap = np.mean(xai_time_shap)
        print('Mean time nedded by appraoches: ' + str(time_mean_lime) + str(time_mean_shap))
        
        if return_errors:
            return auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list, errors
        else:
            return auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list

In [ ]:
model_type = "logit"
train_start = "2013-01-11"
evaluate(df, model_type, train_start, predict_start="2014-01-01", predict_end=-1, return_errors=False,
            weather_sel=False, xai=True)

### **2.4 Pipeline**

Finally, we wrap up all the previously defined functions into the **pipeline** function. This allows to generate a prediction by simply inputting:
* the pre-processed usage data
* the prediction date
* the model type (limited to logit for now)
* the date at which the model has started to train


In [ ]:
def pipeline(self, df, date, model_type, train_start):
    X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)

    # fit model
    model = self.fit(X_train, y_train, model_type)

    # predict
    return self.predict(model, X_test)

# add to Activity agent
setattr(Usage_Agent, 'pipeline', pipeline)
del pipeline

A prediction for the "2013-12-08" based on the data starting on the '2013-11-01' can finally be made for the device with which we initialized the class (here: "Dishwasher")

In [ ]:
date = "2013-12-08"
train_start = '2013-11-01'
usage.pipeline(df, date, 'logit', train_start)

### **2.5 Model Evaluation**

Finally, we want to assess the accuracy of our model before using it in the Recommendation Agent. 

A drawback to our approach is that we are not able to apply conventional model evaluation techniques to our model. We will train our model for each day to account for newly available information. Hence, we have different train and test sets for each day and for each day different performance metric based on the respective data sets. Therefore, we created our own evaluation function. 

Our evaluation function will build a model, fit the model and predict the target for each day for a given prediction period. For each day and fitted model it will calculate a performance metric on the train data. We chose the Area Under the Receiver Operating Characteristic Curve (AUC) as performance metric for our binary classification task. As in our case the test data is only the current date to be predicted, we calculate the test AUC over the usage probabilities of all day after all days have been predicted. To summarize the train AUC in one score, we apply an average over all calculated train AUC scores (Note: This approach is the same as for the activity predictions).

In [ ]:
def auc(self, y_true, y_hat):
    import sklearn.metrics
    return sklearn.metrics.roc_auc_score(y_true, y_hat)

# add to Activity agent
setattr(Usage_Agent, 'auc', auc)
del auc

def evaluate(self, df, model_type, train_start, predict_start='2014-01-01', predict_end=-1):
    import pandas as pd
    import numpy as np
    dates = pd.DataFrame(df.index)
    dates = dates.set_index(df.index)['time']
    predict_start = pd.to_datetime(predict_start)
    predict_end = pd.to_datetime(dates.iloc[predict_end]) if type(predict_end) == int else pd.to_datetime(predict_end)
    dates = dates.loc[predict_start:predict_end]
    y_true = []
    y_hat_train = {}
    y_hat_test = []
    auc_train_dict = {}
    auc_test = []

    for date in dates.index:
        # train test split
        #train_test_split(self, df, date, train_start='2013-11-01', test_delta='all', target='activity')
        X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)

        # fit model
        model = self.fit( X_train, y_train, model_type)

        # predict
        y_hat_train.update({date: self.predict(model, X_train)})
        y_hat_test += list(self.predict(model, X_test))

        # evaluate train data
        auc_train_dict.update({date: self.auc(y_train, list(y_hat_train.values())[-1])})
        
        y_true += list(y_test)
    
    auc_test = self.auc(y_true, y_hat_test)
    auc_train = np.mean(list(auc_train_dict.values()))

    return auc_train, auc_test, auc_train_dict


# add to Activity agent
setattr(Usage_Agent, 'evaluate', evaluate)
del evaluate

Finally, we can evaluate the simple Logit model for the "Dishwasher", for instance for all predictions after the "2014-08-01".  

In [ ]:
auc_train, auc_test, auc_train_dict = usage.evaluate(df, "logit", '2013-11-01', predict_start='2014-08-01', predict_end= -1)
print("mean_auc_on_train = "+ str(auc_train) + " | test_auc = " + str(auc_test))

As can be seen above, the model's performance is quite disappointing. It is not surprising that we do not have a very high accuracy, given the little amount of data we have. However, there must be potential for improvment. A first step in that direction would be a proper feature selection methodology taking into account different devices and households. Moreover, there has been a large decrease in model accuracy after changing the pre-processing pipeline methodology. Therefore, it seems that the model is sensitive to the way we detect the devices' activity. In the next steps we should investigate how and why these pre-processing steps impact the model's performance.

## **Appendix A1: Complete Usage Agent Class**

In [ ]:
class Usage_Agent:
    import pandas as pd

    def __init__(self, input_df, device):
        self.input = input_df
        self.device = device

    # train test split
    # -------------------------------------------------------------------------------------------
    def train_test_split(self, df, date, train_start="2013-11-01"):
        select_vars = [
            self.device + "_usage",
            self.device + "_usage_lag_1",
            self.device + "_usage_lag_2",
            "active_last_2_days",
        ]
        df = df[select_vars]
        X_train = df.loc[train_start:date, df.columns != self.device + "_usage"]
        y_train = df.loc[train_start:date, df.columns == self.device + "_usage"]
        X_test = df.loc[date, df.columns != self.device + "_usage"]
        y_test = df.loc[date, df.columns == self.device + "_usage"]
        return X_train, y_train, X_test, y_test
    
    # model training and evaluation
    # -------------------------------------------------------------------------------------------
    def fit_smLogit(self, X, y):
        import statsmodels.api as sm

        return sm.Logit(y, X).fit(disp=False)

    #####ML Models ##################
    def fit_knn(self, X, y):
        return KNeighborsClassifier(3).fit(X,y)

    def fit_random_forest(self, X,y):
        return RandomForestClassifier(max_depth=5, n_estimators=240, max_features=1).fit(X,y)

    def fit_ADA(self,X,y):
        return  AdaBoostClassifier().fit(X,y)
    def fit_XGB(self, X,y):
        return xgb.XGBClassifier().fit(X,y)
    ##############################################
    def fit(self, X, y, model_type):
        model = None
        if model_type == "logit":
            model = self.fit_smLogit(X, y)
        elif model_type == "ada":
            model = self.fit_ADA(X,y)
        elif model_type == "knn":
            model = self.fit_knn(X,y)
        elif model_type == "random forest":
            model = self.fit_random_forest(X,y)
        elif model_type == "xgboost":
           model = self.fit_XGB(X,y)
        else:
            raise InputError("Unknown model type.")
        return model

    ###############################################

    def predict(self, model, X):
        import numpy as np
        import pandas

        if type(X) == pandas.core.series.Series:
            X = pd.DataFrame(X).transpose() 
        else:
            X=np.squeeze(np.asarray(X))      

        if type(model) == sklearn.linear_model.LogisticRegression:
            y_hat = model.predict(X)
        elif type(model) == sklearn.neighbors._classification.KNeighborsClassifier:
            y_hat = model.predict(X)
        elif type(model) == sklearn.ensemble._forest.RandomForestClassifier:
            y_hat = model.predict(X)
        elif type(model) ==  sklearn.ensemble._weight_boosting.AdaBoostClassifier:
            y_hat = model.predict(X)
        elif type(model) == xgboost.sklearn.XGBClassifier:
            y_hat = model.predict(X)
        else:
            raise InputError("Unknown model type.")
        return y_hat


    def auc(self, y_true, y_hat):
        import sklearn.metrics
        return sklearn.metrics.roc_auc_score(y_true, y_hat)
    
    def evaluate(
        self, df, model_type, train_start, predict_start="2014-01-01", predict_end=-1, return_errors=False
    ):
        import pandas as pd
        import numpy as np
        from tqdm import tqdm

        dates = pd.DataFrame(df.index)
        dates = dates.set_index(df.index)["time"]
        predict_start = pd.to_datetime(predict_start)
        predict_end = (
            pd.to_datetime(dates.iloc[predict_end])
            if type(predict_end) == int
            else pd.to_datetime(predict_end)
        )
        dates = dates.loc[predict_start:predict_end]
        y_true = []
        y_hat_train = {}
        y_hat_test = []
        auc_train_dict = {}
        auc_test = []

        for date in tqdm(dates.index):
            errors = {}
            try:
                X_train, y_train, X_test, y_test = self.train_test_split(
                    df, date, train_start
                )
                # fit model
                model = self.fit(X_train, y_train, model_type)
                # predict
                y_hat_train.update({date: self.predict(model, X_train)})
                y_hat_test += list(self.predict(model, X_test))
                # evaluate train data
                auc_train_dict.update(
                    {date: self.auc(y_train, list(y_hat_train.values())[-1])}
                )
                y_true += list(y_test)
            except Exception as e:
                errors[date] = e

        auc_test = self.auc(y_true, y_hat_test)
        auc_train = np.mean(list(auc_train_dict.values()))

        if return_errors:
            return auc_train, auc_test, auc_train_dict, errors
        else:
            return auc_train, auc_test, auc_train_dict
        
    # pipeline function: predicting device usage
    # -------------------------------------------------------------------------------------------        
    def pipeline(self, df, date, model_type, train_start):
        X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)
        model = self.fit(X_train, y_train, model_type)
        return self.predict(model, X_test)